In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the directory containing the datasets
data_dir = "../../data/"
dataset_path = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa.csv")

In [2]:
data = pd.read_csv(dataset_path)
data.shape

(136400, 21)

In [3]:
data.columns

Index(['category', 'subcategory', 'original_question',
       'original_question_safety', 'original_response',
       'original_response_safety', 'original_response_pre',
       'original_response_pre_safety', 'experiment', 'perturbation_level',
       'perturbation_type', 'perturbation_count', 'perturbed_question',
       'perturbed_question_safety', 'model', 'perturbed_response',
       'perturbed_response_safety', 'perturbed_response_pre',
       'perturbed_response_pre_safety', 'latent_similarity',
       'token_similarity'],
      dtype='object')

In [4]:
# columns with "_pre" in it's name, means questions has been removed from response text. having question in response affect safe of response. that's why question should remove from response.
data.original_response_safety = data.original_response_pre_safety
data.perturbed_response_safety = data.perturbed_response_pre_safety
data.original_response = data.original_response_pre
data.perturbed_response = data.perturbed_response_pre

## 1. Original Question -> Response through all Models -> Labels from Llama Guard

In [5]:
# Count the number of safe and unsafe responses of each model for original questions
data.groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.856364,0.143636
llama3,0.494545,0.505455
llama31,0.670909,0.329091
mistral,0.538182,0.461818


## 2. Original Question -> filter out Unsafe Original Question using LLama-Guard -> Get Response on filtered question from all Models -> Get Labels from Llama Guard

In [6]:
data.original_question_safety.value_counts(normalize=True)

original_question_safety
unsafe    0.932727
safe      0.067273
Name: proportion, dtype: float64

In [7]:
# filtered unsafe original question according to llama guard
data[data["original_question_safety"] == "unsafe"].groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.855750,0.144250
llama3,0.465887,0.534113
llama31,0.656920,0.343080
mistral,0.508772,0.491228


## 3. Original Question -> Do Perturbation(char/word/sntnc) -> Check Safety of Perturbed Question -> Take only unsafe perturbed question -> Response through all Models -> Labels from Llama Guard

### 3.1. Safe percetnage of char, word, sentnc

In [8]:
# Safety percentages for perturbed questions
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    "perturbation_level"
    ).perturbed_response_safety.value_counts(normalize=True).unstack()

perturbed_response_safety,safe,unsafe
perturbation_level,,
char,0.661936,0.338064
sntnc,0.633239,0.366761
word,0.647510,0.352490


### 3.2. re-verification

In [9]:
for level in ["char","word", "sntnc"]:
    dd = data[data.perturbation_level == level]
    dd = dd[dd.perturbed_question_safety == "unsafe"]
    dd = dd.perturbed_response_safety.value_counts(normalize=True)*100
    print(f"{level} : {dd.safe:.2f}")

char : 66.19
word : 64.75
sntnc : 63.32


### 3.3. Unsafe

In [10]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(["model" , "perturbation_level"])["perturbed_response_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_safety        safe     unsafe
model   perturbation_level                      
llama2  char                90.963974   9.036026
        sntnc               83.863636  16.136364
        word                89.131029  10.868971
llama3  char                44.379552  55.620448
        sntnc               49.545455  50.454545
        word                46.065036  53.934964
llama31 char                68.416563  31.583437
        sntnc               65.113636  34.886364
        word                64.441864  35.558136
mistral char                61.014502  38.985498
        sntnc               54.772727  45.227273
        word                59.366034  40.633966

In [11]:
# Set display options
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Do not wrap columns
pd.set_option("display.max_colwidth", None)  # Do not truncate column values

data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["model", "perturbation_level", "category"]
)["perturbed_response_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_safety                                   safe     unsafe
model   perturbation_level category                                        
llama2  char               Adult Content               97.387810   2.612190
                           Child Abuse                 92.328956   7.671044
                           Economic Harm               92.114445   7.885555
                           Fraud/Deception             91.430546   8.569454
                           Hate/Harass/Violence        98.292350   1.707650
                           Illegal Activity            89.869952  10.130048
                           Malware Viruses             77.424893  22.575107
                           Physical Harm               91.146191   8.853809
                           Political Campaigning       90.337079   9.662921
                           Privacy Violation Activity  87.421875  12.578125
                           Tailored Financial Advice   89.260143  10.739857
        sntnc              Adult Content               93.814433   6.185567
                           Child Abuse                 86.046512  13.953488
                           Economic Harm               80.246914  19.753086
                           Fraud/Deception             83.516484  16.483516
                           Hate/Harass/Violence        98.765432   1.234568
                           Illegal Activity            83.870968  16.129032
                           Malware Viruses             59.090909  40.909091
                           Physical Harm               84.883721  15.116279
                           Political Campaigning       91.549296   8.450704
                           Privacy Violation Activity  81.250000  18.750000
                           Tailored Financial Advice   70.312500  29.687500
        word               Adult Content               94.962560   5.037440
                           Child Abuse                 92.257598   7.742402
                           Economic Harm               88.782288  11.217712
                           Fraud/Deception             90.967283   9.032717
                           Hate/Harass/Violence        97.164592   2.835408
                           Illegal Activity            88.666206  11.333794
                           Malware Viruses             75.433790  24.566210
                           Physical Harm               90.063425   9.936575
                           Political Campaigning       87.249399  12.750601
                           Privacy Violation Activity  83.795494  16.204506
                           Tailored Financial Advice   86.042693  13.957307
llama3  char               Adult Content               43.670462  56.329538
                           Child Abuse                 34.968901  65.031099
                           Economic Harm               50.034892  49.965108
                           Fraud/Deception             44.229440  55.770560
                           Hate/Harass/Violence        39.344262  60.655738
                           Illegal Activity            38.329911  61.670089
                           Malware Viruses             41.802575  58.197425
                           Physical Harm               47.975292  52.024708
                           Political Campaigning       40.524345  59.475655
                           Privacy Violation Activity  50.546875  49.453125
                           Tailored Financial Advice   58.711217  41.288783
        sntnc              Adult Content               51.546392  48.453608
                           Child Abuse                 32.558140  67.441860
                           Economic Harm               74.074074  25.925926
                           Fraud/Deception             49.450549  50.549451
                           Hate/Harass/Violence        49.382716  50.617284
                           Illegal Activity            39.784946  60.215054
                           Malware Viruses      

In [12]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["perturbation_level" , "category"]
)["perturbed_response_safety"].value_counts(normalize=True).unstack().T * 100

perturbation_level                 char                            \
category                  Adult Content Child Abuse Economic Harm   
perturbed_response_safety                                           
safe                          70.696584   64.668279     68.702024   
unsafe                        29.303416   35.331721     31.297976   

perturbation_level                                              \
category                  Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                        
safe                            64.789219            67.196038   
unsafe                          35.210781            32.803962   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             58.744011        56.95279     70.075498   
unsafe                           41.255989        43.04721     29.924502   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  61.741573                  68.613281   
unsafe                                38.258427                  31.386719   

perturbation_level                                          sntnc              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                      75.178998     65.979381   59.302326   
unsafe                                    24.821002     34.020619   40.697674   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          68.518519       61.263736             68.82716   
unsafe                        31.481481       38.736264             31.17284   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             53.763441            50.0     75.290698   
unsafe                           46.236559            50.0     24.709302   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  62.676056                       62.5   
unsafe                                37.323944                       37.5   

perturbation_level                                           word              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                        67.1875     67.903336   62.626628   
unsafe                                      32.8125     32.096664   37.373372   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          67.380074       65.131579            65.698479   
unsafe                        32.619926       34.868421            34.301521   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_res